## Importing the libraries

In [1]:
%pip install numpy==1.26.4
%pip install tensorflow

import tensorflow as tf

# Parameters: adjust these as needed
IMG_HEIGHT = 300
IMG_WIDTH = 225
BATCH_SIZE = 32


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Training set

In [ ]:

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../dataset',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../dataset',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

num_classes = len(train_ds.class_names)
print("Detected classes:", train_ds.class_names)


Found 3360 files belonging to 35 classes.
Using 2688 files for training.
Found 3360 files belonging to 35 classes.
Using 672 files for validation.
Detected classes: ['Ё', 'А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', 'Ү', 'Ө']


## CNN Model

In [ ]:

# Define the CNN model
model = tf.keras.models.Sequential([
    # Normalize pixel values to [0, 1]
    tf.keras.layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),

    # Convolutional layers
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),

    # Flatten the results and add a fully connected layer
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    # Output layer: adjust the activation and units based on your task.
    tf.keras.layers.Dense(num_classes, activation='softmax')
])


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## Compiling the model

In [7]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model architecture
model.summary()

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10  # Adjust the number of epochs as needed
)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 300, 225, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 298, 223, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 149, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 147, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 73, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 71, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 35, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 116480)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    14,909,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 35)             │         4,515 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,007,331 (57.25 MB)

 Trainable params: 15,007,331 (57.25 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 178s 2s/step - accuracy: 0.0957 - loss: 3.4321 - val_accuracy: 0.8408 - val_loss: 0.9243
Epoch 2/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step - accuracy: 0.6954 - loss: 1.2159 - val_accuracy: 0.8705 - val_loss: 0.5511
Epoch 3/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 0.8333 - loss: 0.6214 - val_accuracy: 0.8795 - val_loss: 0.5271
Epoch 4/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.8890 - loss: 0.3668 - val_accuracy: 0.8780 - val_loss: 0.5210
Epoch 5/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.9263 - loss: 0.2658 - val_accuracy: 0.8854 - val_loss: 0.5699
Epoch 6/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 186s 2s/step - accuracy: 0.9445 - loss: 0.1858 - val_accuracy: 0.8854 - val_loss: 0.5512
Epoch 7/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 175s 2s/step - accuracy: 0.9544 - loss: 0.1431 - val_accuracy: 0.8914 - val_loss: 0.5466
Epoch 8/10
84/84 ━━━━━━━━━━━━━━━━━━━━ 163s 2s/step - accuracy: 0.9696 - loss: 0.1075 - val_accuracy: 0.8929 - v

## Saving the model

In [9]:
model.save('my_cnn_model.keras')